<a href="https://colab.research.google.com/github/kilianw/Microservices-With-Spring-Student-Files/blob/master/Week_1_PART_2_Function_Approximation_Basics_QUESTIONS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> DUPLICATE THIS COLAB TO START WORKING ON IT. Using File > Save a copy to drive.

# Week 1: Function Approximation Basics: Regression

### Function Approximation: Your new superpower
This project introduces the initial, core set of tools for supervised machine learning (ML). We will use simple tools to load and preprocess datasets in order to focus on the mechanics of creating and using ML models. In this first project, our dataset will be more of a "black box," and later projects will bring together datasets with business context, data cleaning, and ML modeling choices.

Our project will walk through problem formulation, developing models, and evaluating model performance for _regression_ (supervised learning with a continuous output value). We will introduce the basics of formalizing supervised learning as a task along with models and metrics for regression.

Throughout the course, we will build and test models using `scikit-learn` and supporting tools. These tools are often the default starting point for data scientists and ML engineers doing high-stakes analysis or building towards production applications of ML models. The thought process and ML development steps in this project also apply to larger, more complex ML systems. _The core steps we introduce in the project are the same steps you will use for most new supervised learning projects!_

### Instructions

1. This is the ONLY required notebook for Project 1. Other notebooks are optional to help you learn basics, or go deeper. You should complete and submit this notebook for review.
2. We provide starter code and data to give your work a common starting point and scaffolding. You must keep function signatures unchanged to support any later usage and to ensure correct grading of your project. We mark blocks where you need to add code, and you should not need to modify starting code we provide.
3. Ensure you read through the document and starting code before beginning your work. Understand the overall structure and goals of the project to make your implementation efficient.


# Dependencies

Import all the necessary libraries we need throughout the project.

In [ ]:
import pandas as pd
import numpy as np
from typing import Tuple
from sklearn.model_selection import train_test_split
from sklearn import linear_model, svm
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from numpy.core.numeric import Inf
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [10, 5]

# Testing
%pip install -U ipytest
import ipytest
import pytest
ipytest.autoconfig()

Set the random seed for NumPy so that we produce consistent results that can be easily discussed.

In [ ]:
# Fix the random seed so that we get consistent results
np.random.seed(0)

# Predicting diamond price with regression models

Now let's connect the supervised ML formal task definitions to how we build ML models in code. We will introduce regression modeling using a simple dataset from [OpenML](https://www.openml.org/search?type=data&sort=runs&id=42225&status=active). We will use the dataset, called `diamonds`, to predict the prices of diamonds ($y$) from its qualities ($x$). We formalize this regression task as:
* Output $y$: Price of a diamond
* Inputs (also called *features*) $x$: 6 attributes describing a diamond, including carat weight, dimensions, etc.

Now we will load the dataset and build an initial model $f_θ(x) → y$.

## Loading the diamonds  dataset

The dataset is stored in Google Drive. We will download the dataset using `gdown` and load it from a `.npz` file.

In [ ]:
!gdown 1lrU9Uodb09RgnVkNEYDhe4OwEb1E2VIN

In [ ]:
with np.load('diamonds.npz') as npdata:
  X = npdata['X']
  y = npdata['y']

## Basic dataset information

First, if you aren't already familiar with a dataset as a _matrix_, let's look! A dataset is collection of examples. Each example is represented by a set of numbers in an _input vector_ or a _feature vector_. We represent a collection of vectors as a 2-D _matrix_ which is the same as a table you might see in a spreadsheet.

Recall that we are trying to build an ML model $f_θ(x) → y$ where the input $x$ is a vector (a list of numbers). We can then represent a dataset as a matrix by simply "stacking" the input vectors ($x$) together.

Let's look at the diamond dataset as a matrix by checking its shape. The input/feature matrix $X$ has 5000 examples each with an input vector of length 6. The 6 elements in the vector correspond to the 6 features that describe the attributes of the diamond. The output values are stored in $y$ as a vector of 5000 since there is only one output value per example.

In [ ]:
X.shape

In [ ]:
y.shape

Let's quickly check some statistics and basic facts about the dataset which are important for supervised learning. First, let's check the examples and features in our dataset. When working with data for ML tasks, we typically desire each feature variable to have a mean of 0 and a standard deviation of 1. For this project, we have already preprocessed the diamond dataset for you, so the features' means and standard deviations approximately match these values.

In [ ]:
# Check range, min/max, mean, std, etc. for all input feature columns
pd.DataFrame(data=X).describe()

For regression, we typically map a real-valued input vector to a real-valued scalar (or vector). Remember that a scalar is a single value and a vector has multiple values. Many of the techniques we discuss here extend to _multi-variate_ regression where $y$ is a vector.

When working with a dataset, we typically want to understand the rough *distribution* of the output variable $y$. First, let's plot the values of the output variable for our data.

In [ ]:
# scatterplot of y values with jitter
plt.boxplot(y)

**NOTE**: This dataset is already sufficiently pre-processed to use directly with our ML modeling algorithms. Typically when preparing a dataset for ML modeling we need to scale the values of the input and output vectors. In future datasets and projects we will learn about and apply such pre-processing and dataset standardization.

# Building and evaluating a first regression model

We have now loaded the dataset and performed some basic data checks. Note that we are skipping dataset definition and data cleaning to focus this initial project on ML modeling work only.

We will first establish metrics for our model before trying to improve it. This approach ensures that we are clear about what we are measuring before spending time building and improving models.

Some setup steps:
1. Evaluation metric selection and evaluation code.
2. Split data into training / development sets.
3. Build and evaluate simple baselines.

And with our baselines established, we will move to iterative, hypothesis-driven modeling improvements:
1. Build and evaluate a first regression model on the data.
2. Diagnose next steps for how to improve model.
3. Propose hypothesis for how to improve model and iterate!

Let's go!

## Evaluation metric selection

When working with regression, we typically default to mean squared error (MSE) as a guiding metric because it works well with many regression models. However, it's useful to consider multiple evaluation metrics during development so we can monitor for unusual model behavior, and make more comprehensive results comparisons.

 Let's introduce additional metrics to give a different view on performance with less sensitivity to outliers.

To focus our initial work on model development, we will focus on MSE as a standard choice for evaluation, but track the others to check for unusual behavior.


### Evaluation function

To ensure we measure and compare models in a consistent way, we will create a shared function we can use to evaluate all models. Creating and validating this code with a simple baseline before building models gives us a controlled way to test and debug our metrics code before introducing the additional uncertainty of model results.

This function is fairly simple, given actual and predicted values for a regression task, we apply multiple `sklearn.metrics` functions to compute error terms and return 3 values (MSE, MAE, MPE).

In [ ]:
def compute_eval(actual: np.ndarray, pred: np.ndarray ) -> Tuple[float, float, float]:
  """Computes evaluation metrics on given predicted/actual values
  Args:
    actual: 1-D array of actual values
    pred: 1-D array of predicted values

  Returns:
    Mean squared error (float),
    Mean absolute error (float),
    Mean percentage error (float)

  """
  return (mean_squared_error(actual, pred), mean_absolute_error(actual, pred),
   mean_absolute_percentage_error(actual, pred))



### Evaluation unit test

Now let's quickly check our evaluation function using some basic inputs. Think of this as a small software _[unit test](https://en.wikipedia.org/wiki/Unit_testing)_.

In [ ]:
n_samples_test = 100
print("Expect some error (MSE 2.25 MAE 1.5 MPE 1.0)")
print("Computed error: MSE: %.5f  MAE: %.5f  MPE: %.5f "
      % compute_eval(1.5 * np.ones(n_samples_test), np.zeros(n_samples_test)))
print("Expect 0 error")
print("Computed error: MSE: %.5f  MAE: %.5f  MPE: %.5f "
      % compute_eval(np.ones(n_samples_test), np.ones(n_samples_test)))

In these project notebooks, we provide several unit tests using a Python testing framework called `ipytest`. The tests below check your evaluation function much like the cell above, and they output an error message if your code does not pass the tests. You are not required to look at the test code, but you should make sure your tests pass.

In [ ]:
#@title Test: Evaluation function

%%ipytest

@pytest.mark.parametrize('actual, expected', [
    (compute_eval(1.5 * np.ones(n_samples_test), np.zeros(n_samples_test)), (2.25, 1.5, 1.0)),
    (compute_eval(np.ones(n_samples_test), np.ones(n_samples_test)), (0.0, 0.0, 0.0)),
])
def test_compute_eval(actual, expected):
    actual_mse, actual_mae, actual_mpe = actual
    expected_mse, expected_mae, expected_mpe = expected
    assert actual_mse == expected_mse, '''Your MSE calculation is incorrect'''
    assert actual_mae == expected_mae, '''Your MAE calculation is incorrect'''
    assert actual_mpe == expected_mpe, '''Your MPE calculation is incorrect'''


## Split data into training / development sets

Now, we reserve part of the data as a development set to keep fixed during our work. In this case we keep our split simple, we will randomly allocate 80% of the data to a training set, and reserve the remaining 20% for our development set.

Note that we do not have a _test_ set in this project. Remember, a _development_ set is data we withold for regular evaluation of our model during development. A true _test_ set is data we use only once (or rarely) to evaluate a model and get a sense for how it would perform on truly unseen data. We do not need a true test set in this project to learn about building and evaluating models.

In [ ]:
X_train, X_dev, y_train, y_dev = train_test_split(X, y, train_size=0.8, random_state=0)

## **Task: Simple baseline: Predict average value**

With our metrics function validated, let's establish simple baselines on our actual dataset. These baselines are useful for establishing a bound on performance -- a model capturing useful patterns in the data should outperform predicting the average output value for all examples. If models are unable to outperform these baselines, we would expect bugs in our code or issues in the dataset preventing useful learning.

Compute the mean (average) output ($y$) value on the training set. Use this value as a prediction for each dev example. Report results using the evaluation function. Store the mean in a variable called `y_mean`, and store your dev set predictions in a variable called `y_mean_pred`.

In [ ]:
#############################
#### YOUR CODE GOES HERE ####
# Store your answers in the following variables:
# y_mean = ...
# y_mean_pred = ...

#############################

*You should expect a dev set MSE of ~1.051, MAE of ~0.880, and MPE of ~0.114*

In [ ]:
#@title Test: Predict the average value

%%ipytest

def test_mean():
    assert pytest.approx(7.75, 0.02) == y_mean, '''Your mean y value is incorrect'''

@pytest.mark.parametrize('actual, expected', [
    (compute_eval(y_dev, y_mean_pred), (1.0506, 0.8799, 0.1146)),
])
def test_compute_eval(actual, expected):
    actual_mse, actual_mae, actual_mpe = actual
    expected_mse, expected_mae, expected_mpe = expected
    assert pytest.approx(expected_mse, 0.002) == actual_mse, '''Your baseline MSE is incorrect'''
    assert pytest.approx(expected_mae, 0.002) == actual_mae, '''Your baseline MAE is incorrect'''
    assert pytest.approx(expected_mpe, 0.002) == actual_mpe, '''Your baseline MPE is incorrect'''

## Training/fitting a model with scikit


With baselines and evaluation established, it's time to begin building models. Typically we start with a simpler model that is easier to understand. Based on performance of this initial model, we will make decisions about whether to try more complex models on this dataset.

Linear regression is a reliable first choice since it is well-studied, relatively easy to interpret, and has many extensions we can use to improve performance in different situations. Now let's fit a model and generate predictions from it to measure performance relative to actual values.

In [ ]:
## Initialize empty model
regr_lin = linear_model.LinearRegression()

# The .fit() function trains/fits a model using the dataset provided
regr_lin.fit(X_train, y_train)

# Generate predictions using the model
lin_train_pred = regr_lin.predict(X_train)

# Evaluate model predictions vs actual value
print("Training set error: MSE: %.5f  MAE: %.5f  MPE: %.5f "
      % compute_eval(y_train, lin_train_pred))

## **Task: Evaluate initial model**

Above we fit a `LinearRegression` model on the training set, and measured performance of the model on the training set. Now it's your turn! Using the same model we fit above, generate predictions and report performance on the dev set by calling the `compute_eval` function. Store the dev set predictions in a variable `y_dev_lin_pred`.

In [ ]:
#############################
#### YOUR CODE GOES HERE ####
# Store your answer in the following variable:
# y_dev_lin_pred = ...

#############################

*You should expect a dev set MSE of ~0.067, MAE of ~0.202, and MPE of ~0.026.*

In [ ]:
#@title Test: Evaluate initial model

%%ipytest

@pytest.mark.parametrize('actual, expected', [
    (compute_eval(y_dev, y_dev_lin_pred), (0.0666, 0.2018, 0.02618)),
])
def test_compute_eval(actual, expected):
    actual_mse, actual_mae, actual_mpe = actual
    expected_mse, expected_mae, expected_mpe = expected
    assert pytest.approx(expected_mse, 0.002) == actual_mse, '''Your linear model's MSE is incorrect'''
    assert pytest.approx(expected_mae, 0.002) == actual_mae, '''Your linear model's is incorrect'''
    assert pytest.approx(expected_mpe, 0.002) == actual_mpe, '''Your linear model's MPE is incorrect'''

# Use diagnostics to create a hypothesis for how to improve

So far we built and evaluated a first linear model for this task. This process was fairly straightforward, but after fitting an initial model, we have many possible paths for what we might try next.

Let's consider some diagnostic questions to determine the most promising next action to try:
1. Is the model improving compared with simple baselines on both the training and dev sets?
2. Is training set performance at/above goal performance for this system? Is training set performance saturated near zero error?
3. How do training set and dev set results compare with each other?

Think about your response to each question above. Next we will use these diagnostics to decide a next action. Here are reasonable responses to the questions above for our current experiment:
1. _Yes. The model clearly outperforms our simple baselines, indicating the model learned some statistical patterns on the training set which were useful in making correct predictions on the dev set._
1. _Training set performance is far from zero error, and the model does not appear to be fitting training set perfectly._
1. _Training set performance is very close to dev set performance, and not meeting overall performance goals._


**Diagnosis**

Based on our question responses, we might suspect the model is _high bias_ and _low variance_. We conclude this because the model has significant training set error and training/dev errors are closely matched. We also use the term _under-fitting_ to describe a model which performs poorly on the dev set and does not fit the training data as well as we hope.

**Hypothesis**

With our _high bias, low variance, under-fitting_ diagnosis, we can form a hypothesis about how to improve performance. When our model is high bias relative to the task, typically we want to find ways to build a _more expressive model_. In the case of linear regression, the linear assumption of the model might be too simple to fit the training data and capture useful patterns for this problem.

We hypothesize a more expressive model will better fit the training set, and this improved performance will generalize to the dev set. So we want to try building higher variance models.

Let's test this hypothesis by doing a next iteration of our modeling development process with more expressive, nonlinear models.

# Iterating on model improvements: Nonlinear models

## Nearest neighbors model

For a first nonlinear model, we use the simple but powerful _nearest neighbors_. In scikit, this algorithm is available using the same standard regressor interface as our linear model. The class in scikit is`KNeighborsRegressor`.

Nearest neighbors makes a prediction by finding the  $k$-nearest training set examples to an example we wish to make a prediction about. The output of nearest neighbors is usually an average of the training labels from these $k$ examples. This method allows for highly nonlinear predictions since the local average of small collections of points can vary.

We can modify prediction behavior by changing the number of neighbor points and distance or weighting function used to compute predicitons.
Nearest neighbors can be used for both classification and regression. Here the choice of how many neighbors to use ($k$) is a _hyperparameter_ of our model. We use "hyper" because such model options are one level up from the standard model parameters we find from training data using `.fit()`.

For more see [wikipedia](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) or [scikit documentation](https://scikit-learn.org/stable/modules/neighbors.html).

### **Task: Build a K-nearest neighbors model**

Fit a `KNeighborsRegressor` model and report evaluation results on the training and development sets using `compute_eval()`. Adjust model hyperparameter `n_neighbors` ($k$) to find the best dev set performance you can.

**Extension**: If you're already familiar with the basics of using a nearest neighbors model, try exploring more hyperparameters to improve performance. You can refer to documentation for available options, but usually weighting function, distance metric, and number of points are most impactful on predictions. Store your final model in a variable `regr_kn_best`.

In [ ]:
## initialize empty model
regr_kn = KNeighborsRegressor()
# You may initialize a new model to change settings
# Store your final model in a variable:
regr_kn_best = None
#############################
#### YOUR CODE GOES HERE ####

#############################

*You should ensure that your `KNeighborsRegressor` performs better than the linear baseline on the train and dev sets.*

In [ ]:
#@title Test: Build a KNN model

%%ipytest

import sklearn

def test_knn_model():
    assert type(regr_kn_best) == sklearn.neighbors._regression.KNeighborsRegressor, \
    '''Your best model is not a KNeighborsRegressor'''

def test_knn_train_metrics():
    knn_mse, knn_mae, knn_mpe = compute_eval(y_train, regr_kn_best.predict(X_train))
    lin_mse, lin_mae, lin_mpe = compute_eval(y_train, regr_lin.predict(X_train))
    assert knn_mse < lin_mse, \
      '''Your KNN model's training MSE is worse than the linear baseline'''
    assert knn_mae < lin_mae, \
      '''Your KNN model's training MAE is worse than the linear baseline'''
    assert knn_mpe < lin_mpe, \
      '''Your KNN model's training MPE is worse than the linear baseline'''

def test_knn_dev_metrics():
    knn_mse, knn_mae, knn_mpe = compute_eval(y_dev, regr_kn_best.predict(X_dev))
    lin_mse, lin_mae, lin_mpe = compute_eval(y_dev, regr_lin.predict(X_dev))
    assert knn_mse < lin_mse, \
      '''Your KNN model's dev MSE is worse than the linear baseline'''
    assert knn_mae < lin_mae, \
      '''Your KNN model's dev MAE is worse than the linear baseline'''
    assert knn_mpe < lin_mpe, \
      '''Your KNN model's dev MPE is worse than the linear baseline'''

## The importance of train / evaluation data separation

When working in ML we DO NOT call `.fit()` on the development set. Remember we use the development set to approximate how the model will perform on new, unseen data. To see why this is so important, let's see what happens when we allow _data leakage_ -- training the model on data from the development set. The nearest neighbors model stores the data used in `.fit()` to make predictions, so it can very directly show the impact of training on data that we use in evaluation.

### **Task: What happens if you use the development set in training?**

Create a new nearest neighbors model and use the *development* set to fit your model. Report the dev set performance and compare it with the dev set performance you obtained above when properly holding out the development set from training.

You should be able to get nearly perfect performance (0 error) on the dev set by using a small number of points to make each prediction ( controlled with the `n_neighbors` parameters). However, this gives an unrealistic picture of performance, and the dev performance you obtain by training on the dev set is not a good indicator of how the model would perform on new, unseen data. Store this model in a variable `regr_kn_dev`.

**WARNING**: We never train on the development/test set in normal practice. This exercise will show you why we evaluate models on data witheld from training.

In [ ]:
regr_kn_dev = None

#############################
#### YOUR CODE GOES HERE ####
# Store your model in the following variable:
# regr_kn_dev = ...

#############################

In [ ]:
#@title Test: Development set in training

%%ipytest

def test_knn_model():
    assert type(regr_kn_dev) == sklearn.neighbors._regression.KNeighborsRegressor, \
    '''Your model is not a KNeighborsRegressor'''

def test_knn_metrics():
    knn_mse, knn_mae, knn_mpe = compute_eval(y_dev, regr_kn_dev.predict(X_dev))
    assert 0.0 == round(knn_mse, 2), \
      '''Your improperly trained KNN model should have near 0 error'''
    assert 0.0 == round(knn_mae, 2), \
      '''Your improperly trained KNN model should have near 0 error'''
    assert 0.0 == round(knn_mpe, 2), \
      '''Your improperly trained KNN model should have near 0 error'''

We have clear baselines, metrics, and our goal is to find a nonlinear model which improves upon current performance. Let's try it!

We often don't know in advance which type of nonlinear model will perform best on a given dataset. Instead, we focus on making it easy to compare several models and empirically discover which fits our dataset well.

In the section below we will try a few different models to see if they work well on this dataset.

## Decision tree model

Decision trees are a class of models which approximate functions by forming a set of rules or questions about a datapoint. For a quick overview of decision trees you can see the [scikit documentation](https://scikit-learn.org/stable/modules/tree.html). Let's build a first decision tree on our dataset and visualize the model.

In [ ]:
from sklearn import tree

regr_dt = DecisionTreeRegressor()
regr_dt.fit(X_train, y_train)

print("Training set error: MSE: %.5f  MAE: %.5f  MPE: %.5f "
      % compute_eval(y_train, regr_dt.predict(X_train)))

print("Dev set error: MSE: %.5f  MAE: %.5f  MPE: %.5f "
      % compute_eval(y_dev, regr_dt.predict(X_dev)))

### Overfitting the training set and controlling decision tree complexity

Notice the model obtains close to 0 error on the training set -- it fit the training data nearly perfectly! Unfortunately dev set performance is much **worse** than our linear model baseline. This is a classic example of  _overfitting_ the training set which is a technical term in ML for when a model fits the training set well, but learns a function which generalizes poorly to unseen data.

Overfitting happens when our model is too high variance (expressive). It means the model is capable of fitting the training set very well, but fits the training set by finding a function that happens to only fit our training set well rather than all data we would feed to the model. We correct for overfitting by choosing simpler models, or biasing our model search/optimization procedure to prefer less expressive models.

Decision tree models offer a few ways to easily limit model variance / complexity. We can restrict the depth of the decision tree using the `max_depth` parameter, or force the tree building algorithm to only allow leaf nodes when there is some minimum number of training examples corresponding to that leaf (a leaf is the lowest part of the tree after branching based on variable values as we go down the tree from the root) Let's try building a tree with a larger value of `min_samples_leaf`, the default value is 1:

In [ ]:
regr_dt_simpler = DecisionTreeRegressor(min_samples_leaf=20)
regr_dt_simpler.fit(X_train, y_train)

print("Training set error: MSE: %.5f  MAE: %.5f  MPE: %.5f "
      % compute_eval(y_train, regr_dt_simpler.predict(X_train)))

print("Dev set error: MSE: %.5f  MAE: %.5f  MPE: %.5f "
      % compute_eval(y_dev, regr_dt_simpler.predict(X_dev)))

### Visualizing decision trees

The visual plot of a decision tree expresses its set of rules for predicting values of new examples. We start at the top (root) of the tree, and check the value of the variable specified. We choose the left or right branch and repeat this process of checking values and branching left/right until we reach the bottom (leaf) of the tree. The model's predicted value is whatever that leaf specifies.

The model plot is quite dense, don't worry about trying to read specifics of this model. This should give you a tangible sense of "model complexity" and some intuitions for how complex models can effectively memorize a training set in ways that don't generalize well to new data.

OPTIONAL: If you adjust the parameters used in your decision tree you can visually observe differences in the complexity of the resulting tree.

In [ ]:
tree.plot_tree(regr_dt_simpler)

### Model selection helper function

To find good settings for your nearest neighbors model, you likely ran multiple iterations of fitting the model and evaluating results. Overfitting with the decision tree shows why we need to carefully search over model parameters to build a good model.

Let's search the space of possible models in a more principled way. We often frame the problem of ML model development as searching for the best model out of infinitely many possibilities.

Below we provide a function which takes a list of models along with training and dev data. This function calls `.fit()` on each model with the training set, and then evaluates on both training and dev sets. The function returns a list of training and dev metric tuples corresponding to the model list input. This helper function allows us to quickly evaluate a set of candidate models as we search for the best model. It's okay if you can't follow all the Python code here.

In [ ]:
def search_models_helper(model_list: list, X_tr: np.ndarray, y_tr: np.ndarray,
                        X_dev: np.ndarray, y_dev: np.ndarray) -> Tuple[list, list, int]:
  """Evaluate each model in model_list
  Args:
    model_list: 1-D array of scikit models to train with .fit()
    X_tr: 2-D array of training set inputs
    y_tr: 1-D array of training set labels
    X_tr: 2-D array of dev set inputs
    y_tr: 1-D array of dev set labels

  Returns: Tuple() of:
    dev_metrics_list: List of tuples for dev set metrics. Same order as model_list
    train_metrics_list: List of tuples for training set metrics. Same order as model_list
    best_mse_index: int model_list index of lowest dev set MSE model
  """
  best_mse = Inf
  best_idx = -1
  dev_metrics_list = []
  train_metrics_list = []
  for cur_idx, cur_model in enumerate(model_list):
    cur_model.fit(X_tr, y_tr)

    # use evaluation function
    train_metrics_list.append(compute_eval(y_tr, cur_model.predict(X_tr)))
    dev_metrics_list.append(compute_eval(y_dev, cur_model.predict(X_dev)))

    # check if new best model
    cur_mse = dev_metrics_list[-1][0]
    if cur_mse < best_mse:
      best_idx = cur_idx
      best_mse = cur_mse

  return (dev_metrics_list, train_metrics_list, best_idx)

### **Task: Try several decision tree models to obtain better performance**

Below is an example of using the model selection helper to try different settings for the decision tree model. Modify the list of models to try a larger set of possible decision tree models. Store your best model in the `regr_dt_best` variable.

Expected result: In our experiments the decision tree model tends to perform worse on the dev set than our linear baseline. You should try some additional models and achieve dev set MSE worse than the linear baseline, but better than our baseline of predicting the mean value.

In [ ]:
## Store your best model in this variable:
regr_dt_best = None
# List of models to try. Add more models to explore hyperparameter settings.
model_list = []
model_list.append(DecisionTreeRegressor(max_depth=2))
model_list.append(DecisionTreeRegressor(max_depth=10, min_samples_split=50))
#############################
#### YOUR CODE GOES HERE ####

#############################

*You should ensure that your decision tree model has better training set performance than the linear baseline.*

In [ ]:
#@title Test: Decision tree modeling

%%ipytest

def test_dt_model_class():
    assert type(regr_dt_best) == sklearn.tree._classes.DecisionTreeRegressor, \
    '''Your best model is not a DecisionTreeRegressor'''

def test_dt_train_metrics():
    dt_mse, dt_mae, dt_mpe = compute_eval(y_train, regr_dt_best.predict(X_train))
    lin_mse, lin_mae, lin_mpe = compute_eval(y_train, regr_lin.predict(X_train))
    assert dt_mse < lin_mse, \
      '''Your DT model's training MSE is worse than the linear baseline'''
    assert dt_mae < lin_mae, \
      '''Your DT model's training MAE is worse than the linear baseline'''
    assert dt_mpe < lin_mpe, \
      '''Your DT model's training MPE is worse than the linear baseline'''

## **Optional: Try additional nonlinear algorithms**

Nearest neighbors is an intuitive ML algorithm since predictions are based on values from the training set near a query point. Below are some additional modeling approaches that you can try, but you might need to understand more about how the models work to achieve good results. To learn the intuition for any model listed here, you can start with the scikit documentation.

See if you can obtain better dev set performance with a nonlinear model we have not yet tried. Use the search models helper function to try several hyperparameter settings. Report the lowest dev MSE you find and describe the model settings which produced it.

In [ ]:
# Random forest
# Try adjusting n_estimators and max_leaf_nodes
# from sklearn.ensemble import RandomForestRegressor
# RandomForestRegressor(n_estimators=20, max_leaf_nodes=2)

# Gradient boosting regressor
# Try adjusting n_estimators and max_leaf_nodes
# from sklearn.ensemble import GradientBoostingRegressor

# Multi-layer perceptron (Simple neural network)
# Try adjusting hidden_layer_sizes
# from sklearn.neural_network import MLPRegressor

#############################
#### YOUR CODE GOES HERE ####

#############################

## **Plotting performance as a function of model parameters**

It's sometimes helpful to create a plot to show the comparative MSE performance of several models. Here we demonstrate how to plot the performance of nearest neighbors models as we vary $k$, the number of neighbors used.

In [ ]:
# create a list of nearest neighbor models
kn_choices = range(2,50,2)
model_list = []
for k in kn_choices:
  model_list.append(KNeighborsRegressor(n_neighbors=k))

dev_metrics_list, train_metrics_list, best_idx = search_models_helper(model_list, X_train, y_train, X_dev, y_dev)

# plot resulting performance
plt.plot(kn_choices, [ev[0] for ev in dev_metrics_list], 'kx', label='Dev')
plt.plot(kn_choices, [ev[0] for ev in train_metrics_list], 'ko', label='Train')
plt.title('MSE for Nearest Neighbor Models')
plt.xlabel('n_neighbors')
plt.ylabel('MSE')
plt.legend()

### **Optional: Plot performance of models you searched over**

Create a new plot to display performance vs hyperparameter values for any of the other nonlinear models you tried. This can help build intuition for how different options in a nonlinear model affect train and dev set performance.

In [ ]:
#############################
#### YOUR CODE GOES HERE ####

#############################

## **Optional: Hyperparameter search and grid search**

For a more advanced approach to searching for good hyperparameter settings, try defining your list of potential models using a principled hyperparameter search technique. For example, you can specify the minimum and maximum values for each hyperparameter, and then try models for many combinations of values within the ranges. Choosing values at fixed intervals is simple and often effective, it's called _grid search_. You can also try more sophisticated hyperparameter optimization algorithms like [hyperopt](http://hyperopt.github.io/hyperopt/), however these are mostly used when trying to build large models or fully automate model selection.

Below we show an example of using sckit gridsearch for a decision tree model. You can read scikit's [best practices page](https://scikit-learn.org/stable/modules/grid_search.html) for more on how you might set up search for a new task.

In [ ]:
from sklearn.model_selection import GridSearchCV

regr_dt = DecisionTreeRegressor(random_state=0)
param_grid = {'max_depth': [2, 5, 10],
              'min_samples_split': [2,10,30,50]}
dt_gs = GridSearchCV(regr_dt, param_grid, cv=5).fit(X_train, y_train)
print(dt_gs.best_estimator_)

#############################
#### YOUR CODE GOES HERE ####

#############################

# Diagnose next steps after nonlinear modeling improvements

After iterating on nonlinear modeling improvements, let's compare results of the best nonlinear model you found with our initial linear model from earlier. We will evaluate our hypothesis that a higher variance, nonlinear model can better fit the underlying function corresponding to this dataset.

## **Task: Select your best model**

For the various nonlinear models you tried above, choose one that achieves the best dev set error you were able to obtain. Ensure you fit it to the training set, and store it in `regr_nl_best`.

In [ ]:
regr_nl_best = None
#############################
#### YOUR CODE GOES HERE ####

#############################

In [ ]:
#@title Test: Select your best model

%%ipytest

import sklearn

def test_nl_model():
    assert regr_nl_best is not None and \
      type(regr_nl_best) != sklearn.linear_model._base.LinearRegression, \
    '''Your best model is not a KNeighborsRegressor'''

def test_nl_train_metrics():
    nl_mse, nl_mae, nl_mpe = compute_eval(y_train, regr_nl_best.predict(X_train))
    lin_mse, lin_mae, lin_mpe = compute_eval(y_train, regr_lin.predict(X_train))
    assert nl_mse < lin_mse, \
      '''Your nonlinear model's training MSE is worse than the linear baseline'''
    assert nl_mae < lin_mae, \
      '''Your nonlinear model's training MAE is worse than the linear baseline'''
    assert nl_mpe < lin_mpe, \
      '''Your nonlinear model's training MPE is worse than the linear baseline'''

def test_nl_dev_metrics():
    nl_mse, nl_mae, nl_mpe = compute_eval(y_dev, regr_nl_best.predict(X_dev))
    lin_mse, lin_mae, lin_mpe = compute_eval(y_dev, regr_lin.predict(X_dev))
    assert nl_mse < lin_mse, \
      '''Your nonlinear model's dev MSE is worse than the linear baseline'''
    assert nl_mae < lin_mae, \
      '''Your nonlinear model's dev MAE is worse than the linear baseline'''
    assert nl_mpe < lin_mpe, \
      '''Your nonlinear model's dev MPE is worse than the linear baseline'''

## Nonlinear modeling hypothesis evaluation. Did it work?

In [ ]:
print("Linear model:: ")
print("Training set error: MSE: %.5f  MAE: %.5f  MPE: %.5f "
      % compute_eval(y_train, regr_lin.predict(X_train)))

print("Dev set error: MSE: %.5f  MAE: %.5f  MPE: %.5f "
      % compute_eval(y_dev, regr_lin.predict(X_dev)))

print("Best nonlinear model:: ")
print("Training set error: MSE: %.5f  MAE: %.5f  MPE: %.5f "
      % compute_eval(y_train, regr_nl_best.predict(X_train)))

print("Dev set error: MSE: %.5f  MAE: %.5f  MPE: %.5f "
      % compute_eval(y_dev, regr_nl_best.predict(X_dev)))


**Results check**: You should have found at least one nonlinear model which obtains lower dev set MSE than the linear model baseline. For reference, a nearest neighbors model with good setting for $k$ can obtain $ MSE < 0.065$ on the dev set.

Recall our hypothesis that _a more expressive model will better fit the training set, and this improved performance will generalize to the dev set._ In terms of MSE we have validated this hypothesis! Now we have a nonlinear model which generates better aggregate predictions than our initial model. Let's evaluate predictions from our nonlinear model to help form a hypothesis for how to next improve our system.

## Visualizing linear vs nonlinear model predictions

Some additional plotting and diagnostics can help us compare performance between models.
First we can check the _predicted_ vs _actual_ values for $y$ for both models.

### Predicted vs actual values

This scatter plot shows us where model predictions disagree with ground truth, perfect predictions are along the diagonal line $y=x$. This plot helps us check for patterns of errors.

In [ ]:
plt.plot(y_dev, regr_lin.predict(X_dev), 'bo', alpha=0.2, label='linear')
plt.plot(y_dev, regr_nl_best.predict(X_dev), 'ro', alpha=0.2, label='nonlinear')
plt.plot(y_dev, y_dev, 'k-', label='correct')
plt.ylim(np.min(y_dev), np.max(y_dev))
plt.xlabel("Actual ")
plt.ylabel("Predicted")
plt.title('Predicted vs Actual Dev Set')
plt.legend()
plt.show()

**Results check**: Depending on the nonlinear model you chose, you might observe more or less systematic differences in the prediction comparison plot. Generally on this dataset you should be able to obtain a nonlinear model which performs better in aggregate than a nonlinear model, and does not exhibit clearly biased predictions in the comparison plot.

### Model prediction disagreement

To more directly understand where predictions differ, we can construct a similar plot using the linear model's predictions as $x$ axis, and your nonlinear model's predictions as $y$ axis. This plot helps us investigate the question, _"How do predictions from the nonlinear model deviate from linear model predictions?_"

* Note this does not say whether either model is more correct, just where the predictions differ.
* The line $y=x$ in this plot is where both models predict the same value for a dev set example.
* Points above the line $y=x$ indicate the nonlinear model predicting a larger value compared with the linear model.
* This diagnostic can help us check for unusual bias or outliers in a more complex model. This helps guide decisions about which model to use in a production system.
* If a nonlinear model is better overall but exhibits non-uniform over/under-estimation compared with the linear model, you might want to further investigate nonlinear model predictions before putting the model into production


In [ ]:
plt.plot(regr_lin.predict(X_dev), regr_nl_best.predict(X_dev), 'bo', alpha=0.2)

plt.plot(regr_lin.predict(X_dev), regr_lin.predict(X_dev), 'k-', label='$y=x$')
plt.ylim(np.min(y_dev), np.max(y_dev))
plt.xlabel("Linear")
plt.ylabel("Nonlinear")
plt.title('Comparing predicted values $y$')
plt.legend()
plt.show()

# Conclusion

Let's review what we have learned from our ML experiments thus far:

* Nonlinear models yielded some improvement on the development set, and our visualizations suggest that nonlinear models are making more accurate predictions without obviously biased behavior compared with the linear model baseline.
* Many nonlinear models can achieve low error on the training set, but improvements do not generalize to the dev set. _Often this is an indicator that the training dataset might not have enough examples or overall variation to warrant more complex models_.
* We coarsely explored different types of nonlinear models, and we can likely further tune our best nonlinear models if we want to improve performance a bit further

**Diagnosis**: Reaching diminishing returns of performance improvement as we try different nonlinear model choices. This is a situation we frequently encounter in ML modeling work: nonlinear models are able to fit the variation in our training set and achieve low/no error on the training set, however the nonlinear models fail to perform well on the development set which suggests the more complex learned function is finding artifacts of the training set rather than useful, generalizable variations in the data. _Overall, we conclude in this situation that our nonlinear models are helping a bit, but we are nearing the end of improvements we can achieve by model changes. Nonlinear models might already be fitting artifacts in the training set and failing to generalize beyond our current performance._

**A note on regularization**: Nonlinear models are _high variance_ and can approximate complex functions. We needed to slightly _reduce_ the variance of our models by choosing simpler versions of nonlinear models to avoid overfitting. Biasing our model search to prefer simpler models is a form of _regularization_. While the math varies depending on the models used, regularization broadly refers to biasing our model search or optimization procedure to prefer certain types of models -- often simpler models. Often times regularization is especially important on smaller datasets, or when the number of input features is large compared with the number of training examples available. We will further explore regularization in upcoming projects.

_Congratulations on completing the project!_ You tried several nonlinear regression models and built a complete initial ML modeling experiment. In this project, we introduced several regression techniques, error metrics, hypothesis-driven development, and regularization.

# Optional: Creating derived features and linear regression regularization

## Improving a linear model with derived features

If a linear model is not sufficiently fitting our training set, we might improve by introducing _new input features_. Adding new input features allows us to fit new linear models on an expanded feature set which hopefully better fit the data.

We create a new input feature vector $\tilde{x} = [x, x_{new}]$ which includes our original features $x$ as well as new features $x_{new}$. Adding new input features often happens as we add new information to improve predictions when _under-fitting_. In ML engineering work, we often create new features by understanding a task or domain, and ensuring all relevant information is available to the ML model (e.g. by having insights on diamond quality and ensuring we have those attributes as inputs to a model).

Rather than expanding $x$ with new information about the task, for now we will make our linear model more expressive by expanding $x$ with _derived features_.
Our new input features will be nonlinear transformations of the existing $x$. Scikit provides a simple _transform_ mechanism which produces new input features $\tilde{x}$ for all examples.

In [ ]:
# perform a polynomial features transform of the dataset
trans = PolynomialFeatures(degree=2)
X_train_trans = trans.fit_transform(X_train)
X_dev_trans = trans.fit_transform(X_dev)

A degree 2 polynomial transform means if we had original input features $x=[a, b]$, the derived input features are $\tilde{x} = [1, a, b, a^2, ab, b^2]$. We can extend this to any polynomial order >= 2, but this can quickly grow the number of features too large for easy use.


Recall a linear model takes the form $y = \theta_1 x_1 + \theta_2 x_2 ...$ so applying a linear model to our derived features allows us to model some nonlinear variation in our original input data because we now have a _linear_ model applied to _nonlinear_ derived features of the data.

### **Optional: Evaluate and compare a linear model with derived features**

Fit and evaluate a linear regression model on the transformed training and dev sets just created.

In [ ]:
# fit an evaluate a linear model on the transformed features
regr_poly_lin = linear_model.LinearRegression()

#############################
#### YOUR CODE GOES HERE ####

#############################

The expanded features model has _lower bias_ and _higher variance_ compared to the model with original features. We conclude this because the new model has more free parameters ($\theta$), and can model some nonlinear relations in the data via the derived features.

## Linear regression with regularization

_Regularization_ introduces a penalty for model complexity when fitting/training the model. By introducing such a penalty, regularization provides a way to trade off model complexity (variance) with how well a model generalizes to perform well on dev set examples.

We typically add a regularization term to the loss function so that our final model will be influenced by the regularization penalty and our original objective/loss function. We now think of our overall loss function $\mathcal{L}_{\mathcal{D}}^{tot}$ as the sum of our original loss function and a new loss term $\mathcal{L}_{\mathcal{D}}^{reg}$,

$\mathcal{L}_{\mathcal{D}}^{tot} (f_\theta(x)) =
\mathcal{L}_{\mathcal{D}}^{fit} (f_\theta(x))
+ \alpha \mathcal{L}_{\mathcal{D}}^{reg} (\theta)$.

The parameter $\alpha$ allows us to adjust the trade-off between the two terms. Directly controlling this tradeoff allows us to penalize the complexity of an over-fitting model and hopefully improve dev set performance by preventing the model from fitting unhelpful relationships in the training set.

An often used, simple regularization approach is sum of squared values of the model parameters $\theta$,

$\mathcal{L}_{\mathcal{D}}^{reg} (\theta) = \alpha \sum_j \theta_j^2$

This sum of squares regularization is also called $L_2$ since it corresponds to the $L_2$ norm of the vector $\theta$.

Finally, let's see our full linear regression loss with $L_2$ regularization,

$\mathcal{L}_{\mathcal{D}} (f_\theta(x)) = \sum_{i \in \mathcal{D}} (\hat{y}^{(i)} - y^{(i)})^2 +
\alpha \sum_j \theta_j^2$

This regression and regularization combination is well-studied and widely used. In fact, it has a special name, ridge regression.


### **Optional: Find a good regularization setting**

Using the `Ridge` model and your `search_models_helper()` function, try several settings for the regularization parameter $\alpha$. Report training and dev set metrics for the best model you find for *both* the expanded features and original datasets.

Fit and store the best model for each feature set. Report the training and dev set metrics of the best model.

Expected result: Regularization should help you improve a bit over the linear baseline model with no regularization.

In [ ]:
# try at least these values but you can add more
alpha_list = [0, 0.01, 0.1, 1, 10, 100, 1000]
## store your lowest MSE model in the variable below
regr_ridge_ = None

#############################
#### YOUR CODE GOES HERE ####

#############################